In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [78]:
#Load files
dem = pd.read_csv('states_demographics.csv')
unemp = pd.read_csv('unemployment_2016_2020.csv')
date = pd.read_csv('date.csv')

In [79]:
dem.head()

,State,effective_date,order_in_place,pop_2020,pop_growth,pop_density,perc_male_pop,median_age_pop,gdp_pc,perc_high_school,perc_bachelors,perc_advanced_degree,billions_state_local_spending
0,Alabama,4/4/20,1,4908621,0.0042,96.9221,0.48,39.2,47735,0.853,0.245,0.091,54.0
1,Alaska,3/28/20,1,734002,-0.0047,1.2863,0.51,34.6,76220,0.924,0.290,0.104,18.0
2,Arizona,3/31/20,1,7378494,0.0288,64.9550,0.49,37.9,51179,0.821,0.284,0.107,66.7
3,Arkansas,NaN,0,3038999,0.0084,58.4030,0.49,38.3,44808,0.856,0.220,0.079,27.3
4,California,3/19/20,1,39937489,0.0096,256.3727,0.49,36.8,80563,0.825,0.326,0.122,623.4


In [80]:
unemp.head()

,State,Filed week ended,Initial Claims,Reflecting Week Ended,Continued Claims,Covered Employment,Insured Unemployment Rate
0,Alabama,1/6/18,"7,534",12/30/17,"25,295","1,872,646",1.35
1,Alabama,1/13/18,"4,387",1/6/18,"21,752","1,872,646",1.16
2,Alabama,1/20/18,"2,701",1/13/18,"19,083","1,872,646",1.02
3,Alabama,1/27/18,"3,134",1/20/18,"18,672","1,872,646",1.00
4,Alabama,2/3/18,"2,455",1/27/18,"17,600","1,872,646",0.94


In [81]:
date.head()

,State,effective_date,order_in_place
0,Alabama,2020-04-04,1
1,Alaska,2020-03-28,1
2,Arizona,2020-03-31,1
3,Arkansas,NaN,0
4,California,2020-03-19,1


In [82]:
date['effective_date'].fillna(value=pd.to_datetime('19/03/2020'), inplace=True)

In [83]:
#Drop unnecessary columns
unemp.drop(['Reflecting Week Ended', 'Continued Claims', 'Covered Employment', 'Insured Unemployment Rate',], axis=1, inplace = True)
date.drop(['order_in_place'],axis = 1, inplace = True)
#Remove the comma in numbers
unemp[['Initial Claims']] = unemp[['Initial Claims']].apply(lambda x: x.str.replace(',','').apply(pd.to_numeric))
#Rename columns
unemp.rename(columns={'Initial Claims':'initial_claims', 'Filed week ended': 'filed_week_ended'}, inplace=True)



In [84]:
#Merge unemployment filings with the effective dates of stay at home orders
unemp = unemp.merge(date, how = 'right', left_on = 'State', right_on = 'State')
#fill nan in date dataframe with a very early date in order to keep the record
#convert date from string to date time
unemp[['filed_week_ended']] = unemp[['filed_week_ended']].apply(pd.to_datetime)
unemp[['effective_date']] = unemp[['effective_date']].apply(pd.to_datetime)


In [85]:
#Take the difference of weeks between order effective date and filed week
unemp['week_diff'] = (unemp['effective_date'] - unemp['filed_week_ended'])/7
unemp['week_diff'] = unemp['week_diff'] / pd.Timedelta(1, unit='d')

In [86]:
#Keep the records with the timeframe 4 weeks before and after the order
unemp = unemp.loc[unemp['week_diff']<=4, :]

In [87]:
#Join the unemployment filing data with demographic data
unemp = unemp.merge(dem, how = 'right', left_on = 'State', right_on = 'State')
#Calculate the unemployment filing per 10000 people
unemp['claim_per_10000_people'] = unemp['initial_claims']*10000/unemp['pop_2020']

In [88]:
#Get the number of weeks before and after the order date
unemp['week']  = 4
unemp.loc[(unemp.week_diff>=3) & (unemp.week_diff<4), 'week'] = 3
unemp.loc[(unemp.week_diff>=2) & (unemp.week_diff<3), 'week'] = 2
unemp.loc[(unemp.week_diff>=1) & (unemp.week_diff<2), 'week'] = 1
unemp.loc[(unemp.week_diff>=0) & (unemp.week_diff<1), 'week'] = 0
unemp.loc[(unemp.week_diff>=-1) & (unemp.week_diff<0), 'week'] = -1
unemp.loc[(unemp.week_diff>=-2) & (unemp.week_diff<-1),'week'] = -2
unemp.loc[(unemp.week_diff>=-3) & (unemp.week_diff<-2), 'week'] = -3
unemp.loc[(unemp.week_diff>=-4) & (unemp.week_diff<-3), 'week'] = -4

In [89]:
unemp

,State,filed_week_ended,initial_claims,effective_date_x,week_diff,effective_date_y,order_in_place,pop_2020,pop_growth,pop_density,perc_male_pop,median_age_pop,gdp_pc,perc_high_school,perc_bachelors,perc_advanced_degree,billions_state_local_spending,claim_per_10000_people,week
0,Alabama,2020-03-07,1663.0,2020-04-04,4.0,4/4/20,1,4908621,0.0042,96.9221,0.48,39.2,47735,0.853,0.245,0.091,54.0,3.387917,4
1,Alabama,2020-03-14,1819.0,2020-04-04,3.0,4/4/20,1,4908621,0.0042,96.9221,0.48,39.2,47735,0.853,0.245,0.091,54.0,3.705725,3
2,Alabama,2020-03-21,10892.0,2020-04-04,2.0,4/4/20,1,4908621,0.0042,96.9221,0.48,39.2,47735,0.853,0.245,0.091,54.0,22.189531,2
3,Alabama,2020-03-28,80984.0,2020-04-04,1.0,4/4/20,1,4908621,0.0042,96.9221,0.48,39.2,47735,0.853,0.245,0.091,54.0,164.983200,1
4,Alabama,2020-04-04,106739.0,2020-04-04,0.0,4/4/20,1,4908621,0.0042,96.9221,0.48,39.2,47735,0.853,0.245,0.091,54.0,217.452111,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,Wyoming,2020-03-14,517.0,2020-03-28,2.0,NaN,0,567025,-0.0185,5.8400,0.51,38.0,68757,0.928,0.267,0.093,9.9,9.117764,2
337,Wyoming,2020-03-21,3653.0,2020-03-28,1.0,NaN,0,567025,-0.0185,5.8400,0.51,38.0,68757,0.928,0.267,0.093,9.9,64.423967,1
338,Wyoming,2020-03-28,6396.0,2020-03-28,0.0,NaN,0,567025,-0.0185,5.8400,0.51,38.0,68757,0.928,0.267,0.093,9.9,112.799259,0
339,Wyoming,2020-04-04,6543.0,2020-03-28,-1.0,NaN,0,567025,-0.0185,5.8400,0.51,38.0,68757,0.928,0.267,0.093,9.9,115.391738,-1


In [90]:
unemp['post_period'] = 0
unemp.loc[unemp['effective_date_x'] <= unemp['filed_week_ended'], 'post_period'] = 1

In [91]:
#Create a new column for a binary variable indicating whether there is order or not
unemp['order'] = 1
unemp.loc[unemp['State'].isin(['North Dakota','South Dakota','Nebraska', 'Arkansas','Iowa']),'order'] = 0
#Drop those states that didn't implement the order in all the states
unemp = unemp.loc[~unemp['State'].isin(['Wyoming','Utah','Oklahoma']),:]







In [99]:
smf.ols('initial_claims ~ C(order) + C(filed_week_ended) + C(State)', data = unemp).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         initial_claims   R-squared:                       0.598
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     7.489
Date:                Tue, 28 Apr 2020   Prob (F-statistic):           7.73e-30
Time:                        22:23:34   Log-Likelihood:                -4048.2
No. Observations:                 321   AIC:                             8204.
Df Residuals:                     267   BIC:                             8408.
Df Model:                          53                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                -5.85e+04   2.57e+04     -2.272      0.024   -1.09e+05   -7802.092
C(order)[T.1]                                            3240.1696   3.01e+04      0.108      0.914   -5.59e+04    6.24e+04
C(filed_week_ended)[T.Timestamp('2020-02-29 00:00:00')]  3.983e+04   3.28e+04      1.215      0.225   -2.47e+04    1.04e+05
C(filed_week_ended)[T.Timestamp('2020-03-07 00:00:00')]  3.479e+04    3.2e+04      1.087      0.278   -2.82e+04    9.78e+04
C(filed_week_ended)[T.Timestamp('2020-03-14 00:00:00')]  3.618e+04    3.2e+04      1.132      0.259   -2.67e+04    9.91e+04
C(filed_week_ended)[T.Timestamp('2020-03-21 00:00:00')]  9.139e+04    3.2e+04      2.859      0.005    2.85e+04    1.54e+05
C(filed_week_ended)[T.Timestamp('2020-03-28 00:00:00')]  1.559e+05    3.2e+04      4.877      0.000     9.3e+04    2.19e+05
C(filed_week_ended)[T.Timestamp('2020-04-04 00:00:00')]  1.592e+05    3.2e+04      4.982      0.000    9.63e+04    2.22e+05
C(filed_week_ended)[T.Timestamp('2020-04-11 00:00:00')]  1.337e+05    3.2e+04      4.183      0.000    7.08e+04    1.97e+05
C(State)[T.Alaska]                                      -3.045e+04   4.43e+04     -0.687      0.493   -1.18e+05    5.68e+04
C(State)[T.Arizona]                                      1.281e+04   4.59e+04      0.279      0.781   -7.76e+04    1.03e+05
C(State)[T.Arkansas]                                    -5319.7545    2.6e+04     -0.204      0.838   -5.66e+04    4.59e+04
C(State)[T.California]                                   3.485e+05   4.32e+04      8.068      0.000    2.63e+05    4.33e+05
C(State)[T.Colorado]                                    -3686.5045   4.43e+04     -0.083      0.934   -9.09e+04    8.36e+04
C(State)[T.Connecticut]                                  -1.85e+04   4.43e+04     -0.418      0.677   -1.06e+05    6.88e+04
C(State)[T.Delaware]                                    -2.866e+04   4.43e+04     -0.647      0.518   -1.16e+05    5.86e+04
C(State)[T.Florida]                                      6.421e+04   4.59e+04      1.398      0.163   -2.62e+04    1.55e+05
C(State)[T.Georgia]                                      9.768e+04   4.59e+04      2.126      0.034    7232.817    1.88e+05
C(State)[T.Hawaii]                                      -1.643e+04   4.43e+04     -0.371      0.711   -1.04e+05    7.08e+04
C(State)[T.Idaho]                                       -2.361e+04   4.43e+04     -0.533      0.595   -1.11e+05    6.37e+04
C(State)[T.Illinois]                                     5.828e+04   4.32e+04      1.349      0.178   -2.68e+04    1.43e+05
C(State)[T.Indiana]                                      2.609e+04   4.43e+04      0.589      0.557   -6.12

In [97]:

smf.ols('initial_claims ~ order + post_period + order * post_period + week + C(State)', data = unemp).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         initial_claims   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     7.345
Date:                Tue, 28 Apr 2020   Prob (F-statistic):           2.99e-28
Time:                        21:14:06   Log-Likelihood:                -4058.8
No. Observations:                 321   AIC:                             8218.
Df Residuals:                     271   BIC:                             8406.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                   2.922e+04   1.53e+04      1.909      0.057    -910.107    5.94e+04
C(State)[T.Alaska]         -5.217e+04   4.54e+04     -1.148      0.252   -1.42e+05    3.73e+04
C(State)[T.Arizona]        -5493.2858   4.73e+04     -0.116      0.908   -9.87e+04    8.77e+04
C(State)[T.Arkansas]        3026.4077   2.69e+04      0.113      0.910   -4.98e+04    5.59e+04
C(State)[T.California]      2.849e+05   4.46e+04      6.392      0.000    1.97e+05    3.73e+05
C(State)[T.Colorado]        -4.37e+04   4.57e+04     -0.956      0.340   -1.34e+05    4.63e+04
C(State)[T.Connecticut]    -5.852e+04   4.57e+04     -1.280      0.202   -1.49e+05    3.15e+04
C(State)[T.Delaware]       -6.868e+04   4.57e+04     -1.502      0.134   -1.59e+05    2.13e+04
C(State)[T.Florida]         4.591e+04   4.73e+04      0.970      0.333   -4.73e+04    1.39e+05
C(State)[T.Georgia]         7.938e+04   4.73e+04      1.677      0.095   -1.38e+04    1.73e+05
C(State)[T.Hawaii]         -5.645e+04   4.57e+04     -1.235      0.218   -1.46e+05    3.36e+04
C(State)[T.Idaho]          -6.362e+04   4.57e+04     -1.392      0.165   -1.54e+05    2.64e+04
C(State)[T.Illinois]        1.301e+04   4.42e+04      0.295      0.769   -7.39e+04    9.99e+04
C(State)[T.Indiana]        -1.393e+04   4.57e+04     -0.305      0.761   -1.04e+05    7.61e+04
C(State)[T.Iowa]            1.226e+04   2.69e+04      0.457      0.648   -4.06e+04    6.51e+04
C(State)[T.Kansas]          -3.82e+04   4.73e+04     -0.807      0.420   -1.31e+05     5.5e+04
C(State)[T.Kentucky]       -2.011e+04   4.57e+04     -0.440      0.660    -1.1e+05    6.99e+04
C(State)[T.Louisiana]      -2.696e+04   4.57e+04     -0.590      0.556   -1.17e+05     6.3e+04
C(State)[T.Maine]          -4.974e+04   4.73e+04     -1.051      0.294   -1.43e+05    4.34e+04
C(State)[T.Maryland]        -1.37e+04   4.73e+04     -0.289      0.773   -1.07e+05    7.95e+04
C(State)[T.Massachusetts]   6581.4802   4.57e+04      0.144      0.886   -8.34e+04    9.66e+04
C(State)[T.Michigan]        7.353e+04   4.57e+04      1.608      0.109   -1.65e+04    1.64e+05
C(State)[T.Minnesota]      -1.502e+04   4.57e+04     -0.328      0.743   -1.05e+05     7.5e+04
C(State)[T.Mississippi]    -4.304e+04   4.73e+04     -0.909      0.364   -1.36e+05    5.01e+04
C(State)[T.Missouri]        1.826e+04   4.96e+04      0.368      0.713   -7.94e+04    1.16e+05
C(State)[T.Montana]        -4.891e+04   4.54e+04     -1.077      0.283   -1.38e+05    4.05e+04
C(State)[T.Nebraska]       -3772.9673   2.69e+04     -0.140      0.888   -5.66e+04    4.91e+04
C(State)[T.Nevada]          -1.31e+04   4.73e+04     -0.277      0.782   -1.06e+05    8.01e+04
C(State)[T.New Hampshire]  -5.958e+04   4.57e+04     -1.303      0.194    -1.5e+05    3.04e+04
C(State)[T.New Jersey]      1.761e+04   4.4